In [ ]:
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import seaborn as sns
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from scipy.stats import pearsonr, randint, spearmanr, uniform
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.metrics import (make_scorer, max_error, mean_squared_error,
                             median_absolute_error)
from sklearn.model_selection import (BaseCrossValidator, GridSearchCV, KFold,
                                     RandomizedSearchCV)
from tqdm import tqdm
from xgboost import XGBRegressor

In [ ]:
def load_dataset(image, annotation):
    # Load dataset image
    with rasterio.open(image) as src:
        raster = src.read()
        bands = src.count
        labels = src.descriptions
        X = raster.transpose(1, 2, 0).reshape(-1, bands)

    # Load annotation
    with rasterio.open(annotation) as src:
        y = src.read(1).flatten()

    # remove nan values
    mask = ~np.isnan(X).any(axis=1)
    mask = np.logical_and(mask, ~np.isnan(y))
    X = X[mask]
    y = y[mask]

    # prettify labels
    pretty_labels = []
    for label in labels:
        label = label.replace('_', ' ')
        pretty_label = ' '.join([word[0].upper() + word[1:]
                                 for word in label.split()])
        pretty_labels.append(pretty_label)

    return X, y, labels

class EndMemberSplitter(BaseCrossValidator):
    def __init__(self, n_splits=5):
        self.n_splits = n_splits
        self.k_fold = KFold(n_splits=n_splits)

    def split(self, X, y, groups=None):
        for train, test in self.k_fold.split(X, y):
            end_member_train = np.where((y[train] == 0)
                                        | (y[train] == 1))[0]
            yield end_member_train, test

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

# Load the dataset
X, y, labels = load_dataset('fusion.tif', 'plot.tif')

In [ ]:
def similarity_matrix(X, method='spearman'):
    if method == 'spearman':
        similarity = spearmanr(X).correlation
    elif method == 'pearson':
        similarity = pearsonr(X).correlation
    elif method == 'mutual_info':
        similarity = np.full((X.shape[1], X.shape[1]), np.nan)
        for i, band_1 in tqdm(enumerate(X.T)):
            if band_1.std() == 0: continue
            for j, band_2 in enumerate(X.T):
                if band_2.std() == 0: continue
                similarity[i, j] = mutual_info_regression(band_1.reshape(-1, 1),
                                                          band_2)[0]
    else:
        raise ValueError(f'Unknown method: "{method}"')
    
    # Ensure the correlation matrix is normalized and symmetric
    np.fill_diagonal(similarity, 0)
    similarity /= np.nanmax(similarity)
    similarity = (similarity + similarity.T) / 2
    similarity[np.isnan(similarity)] = 1
    np.fill_diagonal(similarity, 1)

    return similarity

def plot_dendrogram(similarity_matrix, labels):
    fig = plt.figure(figsize=(12, 8))
    ax = plt.subplot()

    distance_matrix = 1 - np.abs(similarity_matrix)
    dist_linkage = hierarchy.ward(squareform(distance_matrix))
    dendro = hierarchy.dendrogram(
        dist_linkage, labels=labels, ax=ax, leaf_rotation=90
    )

    return dendro

def hierarchial_dim_red(X, labels, similarity_matrix, threshold=0.2):
    distance_matrix = 1 - np.abs(similarity_matrix)
    dist_linkage = hierarchy.ward(squareform(distance_matrix))

    cluster_ids = hierarchy.fcluster(dist_linkage, threshold, criterion="distance")
    cluster_id_to_feature_ids = defaultdict(list)
    for idx, cluster_id in enumerate(cluster_ids):
        cluster_id_to_feature_ids[cluster_id].append(idx)
    
    selected_features = [v[0] for v in cluster_id_to_feature_ids.values()]
        
    return X[:, selected_features], [labels[i] for i in selected_features]

def pairplot(X, labels, similar_labels):
    similar_indices = [labels.index(label) for label in similar_labels]
    similar_df = pd.DataFrame(X[:, similar_indices], columns=similar_labels)

    sns.pairplot(similar_df)

def get_similar_labels(similarity_matrix, labels, num_labels=None, reversed=False, ignore_ones=False):
    # TODO: remove columns with NaN from result if "ignore_ones", as it suggest high importance for those
    similarity_matrix = similarity_matrix.copy()

    # Fill lower triangle with NaNs
    similarity_matrix[np.tril_indices(similarity_matrix.shape[0], -1)] = np.nan

    # Replace 1s with NaNs if requested
    if ignore_ones:
        similarity_matrix[similarity_matrix == 1] = np.nan
    corr_matrix = pd.DataFrame(similarity_matrix, columns=labels, index=labels)

    # Sort by correlation
    corr_matrix = corr_matrix.abs()
    corr_matrix = corr_matrix.unstack()
    corr_matrix = corr_matrix.sort_values(ascending=reversed)
    print('Most similar pairs:')
    print(corr_matrix.head(num_labels))
    corr_pairs = corr_matrix.index

    corr_columns = []
    redundant_columns = (num for tup in corr_pairs for num in tup)  # flatten
    for column in redundant_columns:
        if column not in corr_columns:
            corr_columns.append(column)
            if num_labels is not None and len(corr_columns) >= num_labels:
                break

    return corr_columns

# Draw interesting plots
similarity = similarity_matrix(X, method='spearman')#  'mutual_info')#  
plt.imshow(similarity)
plot_dendrogram(similarity, labels)
similar_labels = get_similar_labels(similarity, labels, num_labels=5, ignore_ones=True)
pairplot(X, labels, similar_labels)

# Dimensionality reduction
# X, labels = hierarchial_dim_red(X, labels, similarity, threshold=0.2)
plt.show()

In [ ]:
# Define the metrics
metrics = {'max_error': max_error,
           'median_absolute_error': median_absolute_error,
           'mean_squared_error': mean_squared_error}
refit = 'mean_squared_error'  # metric for best model selection

# Define the models and hyperparameter search space
# Possible distributions: expon, gamma, uniform, loguniform or randint
# Random state will be set by sklearn, if scipy >= 0.16 is available
models = {
    XGBRegressor(): {
        'n_estimators': randint(10, 100-10),
        'max_depth': randint(1, 20),
        # 'learning_rate': uniform(0.01, 0.5),
        # 'gamma': uniform(0, 0.5),
        # 'min_child_weight': randint(1, 11),
    },
    RandomForestRegressor(): {
        'n_estimators': randint(1, 100-1),
        'max_depth': randint(1, 20),
        'max_features': randint(1, 11),
        'min_samples_split': randint(1, 11),
        'min_samples_leaf': randint(1, 11),
        'bootstrap': [True, False],
        # 'criterion': ['squared_error', 'absolute_error'  'friedman_mse', 'poisson'],
    },
    ExtraTreesRegressor(): {
        'n_estimators': randint(1, 100-1),
        'min_impurity_decrease': uniform(0, 0.5),
        'criterion': ['squared_error', 'absolute_error'],
    },
}

# Define the search strategy
search = RandomizedSearchCV# GridSearchCV#  
cv = EndMemberSplitter(n_splits=5)#  KFold(n_splits=5)#  
search_kwargs = dict(cv=EndMemberSplitter(5), return_train_score=True, n_iter=100, verbose=1, random_state=0)  # , n_jobs=-1

# Perform hyperparameter optimization using the defined search
scorers = {metric_name: make_scorer(metric, greater_is_better=False) for metric_name, metric in metrics.items()}
best_models = []
cv_results = []
for model, param_distributions in models.items():
    current_search = search(model, param_distributions, 
                               scoring=scorers, refit=refit, **search_kwargs)
    current_search.fit(X, y)

    best_models.append(current_search.best_estimator_)
    cv_results.append(current_search.cv_results_)

In [ ]:
# Set colors for plotting
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

# Plot the results for each parameter of each model
for results, (model, param_distributions) in zip(cv_results, models.items()):
    for parameter in param_distributions.keys():
        # Skip if parameter is not convertible to float
        try:
            X_axis = np.array(results[f"param_{parameter}"].data, dtype=float)
        except ValueError:
            continue

        # Setup plot
        plt.figure(figsize=(13, 13))
        ax = plt.subplot()
        ax.set_xlabel(parameter)
        ax.set_ylabel("score")
        ax.set_ylim(0, 1)

        # Sort results by parameter value
        sort_idx = np.argsort(X_axis)
        X_axis = X_axis[sort_idx]
        results = {key: np.array(value)[sort_idx] for key, value in results.items()}

        # Plot the different metrics
        for (metric_name, scorer), color in zip(scorers.items(), colors):
            for sample, style in (("train", "--"), ("test", "-")):
                sample_score_mean = results[f"mean_{sample}_{metric_name}"] * scorer._sign
                sample_score_std = results[f"std_{sample}_{metric_name}"]
                ax.fill_between(
                    X_axis,
                    sample_score_mean - sample_score_std,
                    sample_score_mean + sample_score_std,
                    alpha=0.1 if sample == "test" else 0,
                    color=color,
                )
                ax.plot(
                    X_axis,
                    sample_score_mean,
                    style,
                    color=color,
                    label=f"{metric_name} ({sample})",
                )

            # Find best score
            best_index = np.nonzero(results[f"rank_test_{metric_name}"] == 1)[0][0]
            best_score = results[f"mean_test_{metric_name}"][best_index] * scorer._sign

            # Plot a dotted vertical line at the best score for that scorer marked by x
            ax.plot(
                [X_axis[best_index]] * 2,
                [0, best_score],
                linestyle=":",
                color=color,
                marker="x",
                markeredgewidth=3,
                ms=8,
            )

            # Annotate the best score for that scorer
            ax.annotate("%0.2f" % best_score, (X_axis[best_index], best_score + 0.005))

            # Show legend with metrics and test/train
            ax.legend(loc="best")
        
        # Show plot
        plt.show()